In [1]:
import pandas as pd
import numpy as np
import random
import os
import time

start_time = time.time()

path = os.path.abspath('.')

simRAO = 1000

nMTCD = 10000

nMTCD_success = 0  # total number of device that transmit success

nMTCD_fail = 0  # total number of device that transmit failed

Backoff = 2  # Backoff Indicator = 20ms

maxTrans = 10  # Maximum number of preamble transmission

N_PRB = 25  # Number of Sidelink Physical Resource Block

N_PRB_arrange = 20  # the number of arranged PRB

N_preamble = 54  # Number of PRACH Preamble

file_path = path + f'/MTCD_data_{nMTCD}.csv'
MTCD_data = pd.read_csv(file_path, index_col=False)

"""
RA_data
    records RA parameter and simulation result of every device
    the parameters required in enB <-> device communication
    Index:
        device_id
    Columns 4:
        RA_init:     Integer,
                     next frame the device send RA request
        RA_success:  Integer,
                     frame the device complete RA procedure
        RA_transmit: Integer,
                     number of the device trying to send RA request
"""
RA_data = MTCD_data[['RA_init', 'RA_success', 'RA_transmit']]

"""
D2D_member
    records D2D parameter of every Member
    also records numerical result for every Member
    the parametes required in Member -> Header communication
    Index:
        device_id
    Columns 7:
        Header:   Integer,
                  the Header of belonging D2D group
                  also indicate the group Header's device_id
        Member:   Boolen,
                  whether the device is a Member
        arrange:  Integer,
                  indicate the beloneing arrange sequence
        request:  Boolen,
                  whether the Member will send D2D request
        response: Boolean,
                  whether the device received its Header Response
        success:  Integer,
                  the frame the device finish D2D transmission (-1: not success yet)
        transmit: Integer,
                  number of the Member trying to send D2D request
"""
D2D_member = pd.DataFrame(
    columns=['Header', 'Member', 'arrange',
             'request', 'response', 'success', 'transmit'],
)

# assign group
D2D_member['Header'] = MTCD_data['clusters']

# assign type
D2D_member['Member'] = (D2D_member.index != D2D_member['Header'])

# assign arrange
for header in D2D_member['Header'].unique():
    members = D2D_member.loc[(D2D_member['Header'] == header) &
                             (D2D_member['Member'])]
    n = 0
    for device_id in members.index:
        D2D_member.at[device_id, 'arrange'] = n // N_PRB_arrange
        n += 1

# initialize request
D2D_member['request'] = np.zeros(nMTCD, dtype=bool)

# initialize response
D2D_member['response'] = np.zeros(nMTCD, dtype=bool)

# initialize success = -1
D2D_member['success'] = np.full(nMTCD, -1)

# initialize transmit
D2D_member['transmit'] = np.zeros(nMTCD, dtype=int)

"""
D2D_group
    records D2D parameter of every Header
    also records numerical result for every Header
    records the parameter required by Header -> Member communication
    Index:
        default sequence
    Columns 5:
        Header:     Integer,
                    device_id of the group Header
        class:      Integer,
                    the barring class of the beloneing D2D group
        N_member:   Integer,
                    the total number of device (member+header) in the group
        N_RA:       Integer,
                    number of RA_request within the group
        arr_seq:    Integer,
                    current arrange sequence
        arr_max:    Integer,
                    maximum arrange number
"""
D2D_group = pd.DataFrame(
    columns=['Header', 'class',
             'N_member', 'N_RA', 'arr_seq', 'arr_max']
)

# initialize Header
D2D_group['Header'] = MTCD_data['clusters'].unique()

# assign Class
N_group = len(D2D_group)
N_class = (N_group // N_preamble) + 1
seq = 0
for group in range(N_group):
    D2D_group.at[group, 'class'] = group // N_preamble

# initialize N_member
D2D_group['N_member'] = D2D_member.groupby(['Header']).size().values

# initialize RA_request
D2D_group['N_RA'] = np.zeros(N_group, dtype=int)

# initialize current arrange sequence
D2D_group['arr_seq'] = np.zeros(N_group, dtype=int)


"""
BS_schedule
    indicate the broadcast information for SIB2
    update every 16 system frame
    Do not require initialize
    Index:
        system_frame
    Columns 2:
        class: Integer,
               the class can initiate RA procedure
"""
BS_schedule = pd.DataFrame(
    columns=['class'],
)

# initialize class
BS_schedule['class'] = [f % N_class for f in range(simRAO)]

In [2]:
import time
start = time.process_time()
# your code here    
print(time.process_time() - start)

2.6679999999945636e-05


In [3]:
# dummy data
D2D_member['request'] = np.random.choice([True, False], size=nMTCD)
D2D_member.sample(5)

,Header,Member,arrange,request,response,success,transmit
6209,140,True,1,True,False,-1,0
4103,407,True,0,True,False,-1,0
546,1,True,0,False,False,-1,0
1191,80,True,0,True,False,-1,0
5150,7,True,1,False,False,-1,0


In [4]:
start = time.process_time()
devices_loc = D2D_member.loc[D2D_member['request']]
print(time.process_time() - start)
devices_loc

0.0022372360000000313


,Header,Member,arrange,request,response,success,transmit
0,0,False,NaN,True,False,-1,0
2,2,False,NaN,True,False,-1,0
3,3,False,NaN,True,False,-1,0
5,5,False,NaN,True,False,-1,0
9,9,False,NaN,True,False,-1,0
...,...,...,...,...,...,...,...
9994,4658,True,1,True,False,-1,0
9995,290,True,2,True,False,-1,0
9996,36,True,1,True,False,-1,0
9997,522,True,2,True,False,-1,0


In [55]:

start = time.process_time()
devices_group = D2D_member.groupby('request').get_group(True)
print(time.process_time() - start)
devices_group

0.0033102160000000325


,Header,Member,arrange,request,response,success,transmit
0,0,False,NaN,True,False,-1,0
3,3,False,NaN,True,False,-1,0
6,6,False,NaN,True,False,-1,0
11,11,False,NaN,True,False,-1,0
13,13,False,NaN,True,False,-1,0
...,...,...,...,...,...,...,...
9985,446,True,2,True,False,-1,0
9986,1893,True,2,True,False,-1,0
9990,99,True,1,True,False,-1,0
9995,290,True,2,True,False,-1,0


In [44]:
members_ = D2D_member.loc[D2D_member['request']].copy()

In [45]:
type(members_)

pandas.core.frame.DataFrame

In [47]:
start = time.process_time()
for header in headers:
    group_id = D2D_group.loc[D2D_group['Header'] == header].index[0]
    cur_arrange = D2D_group.at[group_id, 'arr_seq']
    arr_members = members_.loc[members_.index].loc[(members_.Header == header) & (members_.arrange == cur_arrange)].index
    print(arr_members)
    non_arr_members = members.loc[members_.index.difference(arr_members)].index

print(time.process_time() - start)
    

Int64Index([789, 1213, 1264, 1292, 1496, 1684, 2683, 3042, 3508, 3594, 3782], dtype='int64')
Int64Index([30, 194, 930, 973, 1393, 1497, 1522, 1737, 1780], dtype='int64')
Int64Index([2366, 3057, 3892, 4629], dtype='int64')
Int64Index([261, 874, 2115, 2760, 2765, 2778, 4186, 4531], dtype='int64')
Int64Index([1362, 1712, 1746, 1873, 3190, 3271, 3291, 3506, 3853, 3898], dtype='int64')
Int64Index([74, 802, 1088, 3993, 4724, 4973], dtype='int64')
Int64Index([171, 353, 635, 1856, 1874, 2185, 2320, 4349, 4875, 5466], dtype='int64')
Int64Index([1451, 1608, 2092, 2638, 4066, 5550, 6123, 6267, 6517], dtype='int64')
Int64Index([150, 312, 313, 502, 1477, 1563, 1762, 1861, 3498], dtype='int64')
Int64Index([637, 661, 831, 881, 927, 1190, 1322, 1618, 1643, 1674, 1812, 2228], dtype='int64')
Int64Index([135, 864, 1343, 1899, 1954, 1966, 2085, 2947, 3485, 4855], dtype='int64')
Int64Index([24, 969, 989, 1104, 1313, 1839, 2003, 2463, 2492, 3117, 3633,
            3955],
           dtype='int64')
Int64Index

In [40]:
D2D_member.loc[[0,1,2,3]].loc[D2D_member['request']]

,Header,Member,arrange,request,response,success,transmit
0,0,False,NaN,True,False,-1,0
2,2,False,NaN,True,False,-1,0
3,3,False,NaN,True,False,-1,0
